# Optimizando código en `Python`

More articles in my blog:   $\hspace{0.1cm}$   [Estadistica4all](https://fabioscielzoortiz.github.io/Estadistica4all.github.io/)

&nbsp;

How to reference this article ? 

Scielzo Ortiz, F. (2022). Optimizando código en Python. *Estadistica4all*.  [Optimizando código en Python](https://fabioscielzoortiz.github.io/Estadistica4all.github.io/Articulos/Optimizando%20codigo%20en%20Python.html )

## Index

* [Codigo sin optimizar ](#1)
* [Paralelización de bucles for](#2)
* [Usar `Numpy` en vez de `Pandas`](#3)
* [Alternativas a los bucles for](#4)


---

## Codigo sin optimizar <a class="anchor" id="1"></a>

La justificacion principal de optimizar nuestros codigos en ciencia de datos, poder implementar procedimientos en caso reales que exigen un alto rendimiento computacional.

Si para ejecutar un procedimiento en un caso real el ordenador tarda horas, dias o es incluso intratable computacionalemente, directamente no podremos aplicar dicho procedimiento (por bueno que sea) en casos reales, y tendremos que limitarnos a usar procedimientos quiza peores desde un punto de vista estadistico, pero mejores a nivel de coste computacional.

La idea de optimizar codigo es no tener que desechar buenos procedimientos a nivel estadistico porque sean malos a nivel computacional (sean ineficientes computacionalmente), la idea es hacerlos mas eficientes, para reducir sus costes de computacion.

El siguiente codigo es el que usaremos como ejemplo a lo largo de este articulo, la idea es medir su rendimiento original y posteriormente aplicarle diferentes tecnicas para optimizarlo y ver como cambia su rendimiento.

In [62]:
import pandas as pd
import numpy as np

In [63]:
House_Prices_Data = pd.read_csv('House_Price_Regression.csv')

House_Prices_Data = House_Prices_Data.loc[:, ['price', 'size_in_m_2', 'no_of_bedrooms', 'no_of_bathrooms', 'quality_recode', 'latitude', 'private_garden_recode', 'private_pool_recode', 'longitude']]

House_Prices_Data['quality_recode'] = House_Prices_Data['quality_recode'].astype('object')
House_Prices_Data['private_garden_recode'] = House_Prices_Data['private_garden_recode'].astype('object')
House_Prices_Data['private_pool_recode'] = House_Prices_Data['private_pool_recode'].astype('object')

House_Prices_Data = House_Prices_Data.rename({'price': 'Y'}, axis=1)

Data_Houses_Prices_Train = House_Prices_Data.sample(frac=0.8, replace=False, weights=None, random_state=555, axis=None, ignore_index=True)

Data_Houses_Prices_Test = House_Prices_Data.drop( Data_Houses_Prices_Train.index , )

###########################################################################

# Ahora vamos a sacar por un lado los X (predictores) de test y train , y por otro Y (respuesta) de test y train

# Ademas vamos a generar un nuevo Data_Test=[Y_test, X_tes]  y Data_Train=[Y_train, X_train]

# Con estos data frames son con los que nos vamos a manejar

X_test = Data_Houses_Prices_Test.loc[: , ['size_in_m_2', 'no_of_bedrooms', 'no_of_bathrooms','latitude', 'longitude', 'private_garden_recode', 'private_pool_recode',  'quality_recode']]

Y_test = Data_Houses_Prices_Test.loc[: , 'Y']

Data_Test = pd.concat([Y_test , X_test], axis=1)

#########################

X_train = Data_Houses_Prices_Train.loc[: , ['size_in_m_2', 'no_of_bedrooms', 'no_of_bathrooms','latitude', 'longitude', 'private_garden_recode', 'private_pool_recode',  'quality_recode']]

Y_train = Data_Houses_Prices_Train.loc[: , 'Y']

Data_Train = pd.concat([Y_train , X_train], axis=1)

In [64]:
# Como ejemplo de x_new (nueva observacion de los predictores) cogemos la sexta (5 en python) observacion de X_test

x_new = X_test.iloc[ 5 , range(0, X_test.shape[1])]

In [65]:
X_train_small = X_train.iloc[0:100]
Y_train_small = Y_train.iloc[0:100]

In [66]:
X_train_medium = X_train.iloc[0:500]
Y_train_medium = Y_train.iloc[0:500]

La siguiente funcion es una implementacion de KNN para regresion usando la distancia de Gower, que es una metrica computacionalmente muy costosa.

La idea es usar la funcion KNN_regresion dentro de una rutina de validacion cruzada, como ejemplo de procedimiento que supone un alto coste computacional.

In [67]:
def KNN_regression( X , Y , x_new, k, distance, p1, p2, p3 ):

   
####################################################################################################################################################################################################################################################

    # Y tiene que ser una variable respuesta cuantitativa

    # X tiene que ser un panda data frame con los predictotres (X1,...,Xp). 

    # x_new tiene que ser un vector. 

####################################################################################################################################################################################################################################################

    X  = pd.concat([X , x_new.to_frame().T], ignore_index=True)

    distances = []

    Y_values_knn = []


####################################################################################################################################################################################################################################################
    
    
    
    if distance == "Gower":

        # The data matrix X have to be order in the following way:
        # The p1 first are quantitative, the following p2 are binary categorical, and the following p3 are multiple categorical.


        def a(Binary_Data) :

            X = Binary_Data

            a = X @ X.T

            return(a)

##########################################################################################

        def d(Binary_Data):

            X = Binary_Data

            ones_matrix = np.ones(( X.shape[0] , X.shape[1])) 

            d = (ones_matrix - X) @ (ones_matrix - X).T

            return(d)

##########################################################################################

        def alpha_py(i,j, Multiple_Categorical_Data):

                X = Multiple_Categorical_Data

                alpha = np.repeat(0, X.shape[1])

                for k in range(0, X.shape[1]) :

                    if X.iloc[i-1, k] == X.iloc[j-1, k] :

                        alpha[k] = 1

                    else :

                        alpha[k] = 0


                alpha = alpha.sum()

                return(alpha)

   ##########################################################################################


        def Gower_Similarity_Python(i,j, Mixed_Data_Set, p1, p2, p3):

            X = Mixed_Data_Set

       # The data matrix X have to be order in the following way:
       # The p1 first are quantitative, the following p2 are binary categorical, and the following p3 are multiple categorical.

   #####################################################################################
        
            def G(k, X):

                range = X.iloc[:,k].max() - X.iloc[:,k].min() 

                return(range)

            G_vector = np.repeat(0.5, p1)

            for r in range(0, p1):

                G_vector[r] = G(r, X)
                
      
    ##########################################################################################
    
            ones = np.repeat(1, p1)

            Quantitative_Data = X.iloc[: , 0:p1]

            Binary_Data = X.iloc[: , (p1):(p1+p2)]
            
            Multiple_Categorical_Data = X.iloc[: , (p1+p2):(p1+p2+p3) ]

    ##########################################################################################

            numerator_part_1 = ( ones - ( (Quantitative_Data.iloc[i-1,:] - Quantitative_Data.iloc[j-1,:]).abs() / G_vector ) ).sum() 

            numerator_part_2 = a(Binary_Data).iloc[i-1,j-1] + alpha_py(i,j, Multiple_Categorical_Data)

            numerator = numerator_part_1 + numerator_part_2
 
            denominator = p1 + (p2 - d(Binary_Data).iloc[i-1,j-1]) + p3

            Similarity_Gower = numerator / denominator  

            return(Similarity_Gower)

##########################################################################################

        def Dist_Gower_Py(i, j, Mixed_Data , p1, p2, p3):

            Dist_Gower = np.sqrt( 1 - Gower_Similarity_Python(i, j, Mixed_Data , p1, p2, p3) )

            return(Dist_Gower)    

    ###################################################################

        for j in range(1, len(X)):

            distances.append( Dist_Gower_Py( len(X), j , X, p1, p2, p3) )

        
######################################################################################################################################
    
    distances = pd.DataFrame({'distances': distances})

    distances = distances.sort_values(by=["distances"]).reset_index(drop=False)
        
    knn = distances.iloc[0:k , :]

    for i in knn.iloc[:,0]:

        Y_values_knn.append(Y.iloc[i , ])


    y_predict = sum(Y_values_knn)/k


                                     
    return y_predict   , distances

In [68]:
y_predict  , distances = KNN_regression( X= X_train_small , Y= Y_train_small , x_new=x_new, k=10,  distance="Gower" , p1=5, p2=2, p3=1 )

In [69]:
y_predict

3762000.0

In [70]:
distances

,index,distances
0,97,0.138970
1,16,0.185224
2,46,0.244530
3,49,0.297597
4,58,0.322041
...,...,...
95,94,0.730137
96,13,0.734777
97,63,0.744727
98,69,0.789989


La siguiente rutina de validacion simple usando la funcion KNN_regression es la que usaremos como ejemplo a lo largo de este artuiculo, ahora podremos ver cuanto tarda en ejecutarse con el algoritmo original sin haber sido optimizado de ningun modo, despues repetiremos el proceso habiendo optimizado el algoritmo con diferentes procedimientos:

In [71]:
def validacion_simple(Data_Test, X_train, Y_train):

    ##########################

    def prediction(i, Data_Test, X_train, Y_train ):

     x_new = Data_Test.iloc[ i , range(1, Data_Test.shape[1])]
 
     y_new_predict , distances = KNN_regression( X_train  , Y_train , x_new, k=10, distance = "Gower" , p1=5, p2=2, p3=1  )

     return(y_new_predict)

    ##########################

    y_predictions_vector = []

    for i in  range(0, len(Data_Test)):

        y_new_predict = prediction(i, Data_Test, X_train, Y_train )

        y_predictions_vector.append( y_new_predict )

    ##########################

    ECM = ( (y_predictions_vector - Data_Test.loc[: , 'Y'])**2 ).sum() 

 
    return(y_predictions_vector , ECM)

In [72]:
y_predictions_vector, ECM = validacion_simple(Data_Test=Data_Test, X_train=X_train_small , Y_train=Y_train_small)

In [73]:
ECM

3054953133867935.5

El tiempo de computacion usando solo las 100 primeras filas de X_train e Y_train es de  **5.30 minutos** en media (se ha corrido 4 veces).

In [74]:
# y_predictions_vector, ECM = validacion_simple(Data_Test=Data_Test, X_train=X_train_medium , Y_train=Y_train_medium)

El tiempo de computacion usando las 500 primeras filas de X_train e Y_train **mas de 70 mins** (se paró la ejecucion antes de completarla)

Como se ve, usando 500 filas de X_train e Y_train ya es practicamente imposible aplicar nuestros procedimientos.

Si usasemos X_train e Y_train enteras (que tienen  1524 filas) directamente no seria posinle aplicar el algoritmo. 

---

## Paralelización de bucles for <a class="anchor" id="2"></a>

Usaremos la libreria `joblib` para paralelizar los bucles for del algoritmo KNN_regression, tambien haremos lo mismo con el bloque de codigo con el que hacemos la validacion simple

In [75]:
def KNN_regression_Parallel_Pandas( X , Y , x_new, k, distance  , p1=0, p2=0, p3=0 ):

##########################################################################################################

## Para paralelizar el algoritmo 

    from joblib import Parallel, delayed
    import multiprocessing

####################################################################################################################################################################################################################################################

    # Y tiene que ser una variable respuesta cuantitativa

    # X tiene que ser un panda data frame con los predictotres (X1,...,Xp). 

    # x_new tiene que ser un vector. 

####################################################################################################################################################################################################################################################

    X  = pd.concat([X , x_new.to_frame().T], ignore_index=True)
 
    distances = []
    Y_values_knn = []

####################################################################################################################################################################################################################################################
    
    
    
    if distance == "Gower":

        # The data matrix X have to be order in the following way:
        # The p1 first are quantitative, the following p2 are binary categorical, and the following p3 are multiple categorical.


        def a(Binary_Data) :

            X = Binary_Data

            a = X @ X.T

            return(a)

##########################################################################################

        def d(Binary_Data):

            X = Binary_Data

            ones_matrix = np.ones(( X.shape[0] , X.shape[1])) 

            d = (ones_matrix - X) @ (ones_matrix - X).T

            return(d)

##########################################################################################

        def alpha_py(i,j, Multiple_Categorical_Data):

                X = Multiple_Categorical_Data

                alpha = np.repeat(0, X.shape[1])

                for k in range(0, X.shape[1]) :

                    if X.iloc[i-1, k] == X.iloc[j-1, k] :

                        alpha[k] = 1

                    else :

                        alpha[k] = 0


                alpha = alpha.sum()

                return(alpha)

   ##########################################################################################


        def Gower_Similarity_Python(i,j, Mixed_Data_Set, p1, p2, p3):

            X = Mixed_Data_Set

   # The data matrix X have to be order in the following way:
   # The p1 first are quantitative, the following p2 are binary categorical, and the following p3 are multiple categorical.

   #####################################################################################
        
            def G(k, X):

                range = X.iloc[:,k].max() - X.iloc[:,k].min() 

                return(range)

            G_vector = np.repeat(0.5, p1)

            for r in range(0, p1):

                G_vector[r] = G(r, X)
                
      
    ##########################################################################################
    
            ones = np.repeat(1, p1)

            Quantitative_Data = X.iloc[: , 0:p1]

            Binary_Data = X.iloc[: , (p1):(p1+p2)]
            
            Multiple_Categorical_Data = X.iloc[: , (p1+p2):(p1+p2+p3) ]

    ##########################################################################################

            numerator_part_1 = ( ones - ( (Quantitative_Data.iloc[i-1,:] - Quantitative_Data.iloc[j-1,:]).abs() / G_vector ) ).sum() 

            numerator_part_2 = a(Binary_Data).iloc[i-1,j-1] + alpha_py(i,j, Multiple_Categorical_Data)

            numerator = numerator_part_1 + numerator_part_2
 
            denominator = p1 + (p2 - d(Binary_Data).iloc[i-1,j-1]) + p3

            Similarity_Gower = numerator / denominator  

            return(Similarity_Gower)

##########################################################################################

        def Dist_Gower_Py(i, j, Mixed_Data , p1, p2, p3):

            Dist_Gower = np.sqrt( 1 - Gower_Similarity_Python(i, j, Mixed_Data , p1, p2, p3) )

            return(Dist_Gower)    

    ###################################################################

    ### PARTE DEL CODIGO A PARALELIZAR

        # for i in range(1, len(X)):

            # distances.append( Dist_Gower_Py( len(X), i , X, p1, p2, p3) )


        n_jobs  = multiprocessing.cpu_count()

        distances = Parallel(n_jobs=n_jobs)( delayed(Dist_Gower_Py)( len(X), s , X, p1, p2, p3) for s in range(1, len(X)) )

######################################################################################################################################
    

    distances = pd.DataFrame({'distances': distances})

    distances = distances.sort_values(by=["distances"]).reset_index(drop=False)
        
    knn = distances.iloc[0:k , :]

    for i in knn.iloc[:,0]:

        Y_values_knn.append(Y.iloc[i , ])


    y_predict = sum(Y_values_knn)/k


                                     
    return y_predict  , distances 

In [76]:
y_predict  , distances = KNN_regression_Parallel_Pandas( X=X_train_small , Y=Y_train_small , x_new=x_new, k=10,  distance="Gower" , p1=5, p2=2, p3=1 )

In [77]:
y_predict

3762000.0

In [78]:
distances

,index,distances
0,97,0.138970
1,16,0.185224
2,46,0.244530
3,49,0.297597
4,58,0.322041
...,...,...
95,94,0.730137
96,13,0.734777
97,63,0.744727
98,69,0.789989


In [79]:
def validacion_simple_Parallel_Pandas(Data_Test, X_train, Y_train):

    ##########################

    from joblib import Parallel, delayed
    import multiprocessing

    n_jobs  = multiprocessing.cpu_count()

    ##########################

    def prediction(i, Data_Test, X_train, Y_train ):

     x_new = Data_Test.iloc[ i , range(1,Data_Test.shape[1])]

     # Usamos KNN_regression_Parallel en vez de KNN_regression
 
     y_new_predict , distances = KNN_regression_Parallel_Pandas( X=X_train  , Y=Y_train , x_new=x_new, k=10, distance = "Gower" , p1=5, p2=2, p3=1  )
     

     return(y_new_predict)

    ##########################

    

    # Paralelizamos el siguiente bucle for :

    # for i in  range(0, len(Data_Test)):

        # y_new_predict = prediction(i, Data_Test, X_train, Y_train )

        # y_predictions_vector.append( y_new_predict )

    y_predictions_vector = []

    y_predictions_vector = Parallel(n_jobs=n_jobs)( delayed(prediction)( i, Data_Test, X_train, Y_train) for i in range(0, len(Data_Test)) )

    #########################

    ECM = ( (y_predictions_vector - Data_Test.loc[: , 'Y'])**2 ).sum() 


    return(y_predictions_vector, ECM)

In [80]:
y_predictions_vector , ECM = validacion_simple_Parallel_Pandas(Data_Test=Data_Test, X_train=X_train_small, Y_train=Y_train_small)

In [81]:
ECM

3054953133867935.5

En este caso, tras paralelizar los algoritmos el tiempo de computacion se reduce de **5.30 mins** a **1.35 mins** (en media, se ha corrido 3 veces) , lo cual es bastante.

---

## Usar `Numpy` en vez de `Pandas`  <a class="anchor" id="3"></a>

In [100]:
def KNN_regression_Parallel_Numpy( X , Y , x_new, k, distance, p1=0, p2=0, p3=0 ):

##########################################################################################################

## Para paralelizar el algoritmo 

    from joblib import Parallel, delayed
    import multiprocessing

    n_jobs  = multiprocessing.cpu_count()

####################################################################################################################################################################################################################################################

    # Y, X y x_new deben ser objetos Pandas ya que luego seran convertidos a objetos Numpy automaticamente por el algoritmo
    
    # Y tiene que ser un Pandas data frame con la variable respuesta 

    # X tiene que ser un Pandas data frame con los predictotres (X1,...,Xp). 

    # x_new tiene que ser un vector con una nueva observacion de los predictores. 

####################################################################################################################################################################################################################################################

    Y = Y.to_numpy()

    X = X.to_numpy() 

    x_new = x_new.to_numpy()

    X = np.concatenate((X, [x_new]), axis=0)


    distances = []

    Y_values_knn = []


####################################################################################################################################################################################################################################################
    
    
    
    if distance == "Gower":

        # The data matrix X have to be order in the following way:
        # The p1 first are quantitative, the following p2 are binary categorical, and the following p3 are multiple categorical.


        def a(Binary_Data) :

            X = Binary_Data

            a = X @ X.T

            return(a)

##########################################################################################

        def d(Binary_Data):

            X = Binary_Data

            ones_matrix = np.ones(( X.shape[0] , X.shape[1])) 

            d = (ones_matrix - X) @ (ones_matrix - X).T

            return(d)

##########################################################################################

        def alpha_py(i,j, Multiple_Categorical_Data):

            X = Multiple_Categorical_Data

            alpha = np.repeat(0, X.shape[1])

            def argumento_bucle_for(k):

                if X[i-1, k] == X[j-1, k] :

                    alpha[k] = 1

                else :

                    alpha[k] = 0

                return(alpha) 
    
            alpha=Parallel(n_jobs=n_jobs)( delayed(argumento_bucle_for)( k ) for k in range(0, X.shape[1]) )
    
            alpha = sum(alpha)

            return(alpha)

##########################################################################################

        def Gower_Similarity_Python(i,j, Mixed_Data_Set, p1, p2, p3):

            X = Mixed_Data_Set

   # The data matrix X have to be order in the following way:
   # The p1 first are quantitative, the following p2 are binary categorical, and the following p3 are multiple categorical.

   #####################################################################################
        
            def G(k, X):

                range = X[:,k].max() - X[:,k].min() 

                return(range)

            G_vector = np.repeat(0.5, p1)

            for r in range(0, p1):

                G_vector[r] = G(r, X)
                
      
    ##########################################################################################
    
            ones = np.repeat(1, p1)

            Quantitative_Data = X[: , 0:p1]

            Binary_Data = X[: , (p1):(p1+p2)]
            
            Multiple_Categorical_Data = X[: , (p1+p2):(p1+p2+p3) ]

    ##########################################################################################

            numerator_part_1 = ( ones - ( abs(Quantitative_Data[i-1,:] - Quantitative_Data[j-1,:]) / G_vector ) ).sum() 

            numerator_part_2 = a(Binary_Data)[i-1,j-1] + alpha_py(i,j, Multiple_Categorical_Data)

            numerator = numerator_part_1 + numerator_part_2
 
            denominator = p1 + (p2 - d(Binary_Data)[i-1,j-1]) + p3

            Similarity_Gower = numerator / denominator  

            return(Similarity_Gower)
  
##########################################################################################

        def Dist_Gower_Py(i, j, Mixed_Data , p1, p2, p3):

            Dist_Gower = np.sqrt( 1 - Gower_Similarity_Python(i, j, Mixed_Data , p1, p2, p3) )

            return(Dist_Gower)    

    ###################################################################

    ## PARTE DEL CODIGO A PARALELIZAR

        #for j in range(1, len(X)):

          #distances.append( Dist_Gower_Py( len(X), j , X, p1, p2, p3) )

        n_jobs  = multiprocessing.cpu_count()

        distances = Parallel(n_jobs=n_jobs)( delayed(Dist_Gower_Py)( len(X), s , X, p1, p2, p3) for s in range(1, len(X)) )

######################################################################################################################################
    
    distances = pd.DataFrame({'distances': distances})

    distances = distances.sort_values(by=["distances"]).reset_index(drop=False)
        
    knn = distances.iloc[0:k , :]

    for i in knn.iloc[:,0] :

        Y_values_knn.append(Y[i, ])


    y_predict = sum(Y_values_knn)/k

                                     
    return y_predict  , distances 

In [101]:
y_predict  , distances = KNN_regression_Parallel_Numpy( X=X_train_small , Y=Y_train_small , x_new=x_new, k=10, distance="Gower", p1=5, p2=2, p3=1)

In [102]:
y_predict

3762000.0

In [85]:
distances

,index,distances
0,97,[0.13897014813456796]
1,16,[0.1852237164505301]
2,46,[0.24452975158447923]
3,49,[0.29759738381558937]
4,58,[0.32204114335070677]
...,...,...
95,94,[0.7301372273662897]
96,13,[0.7347771987827965]
97,63,[0.7447272350811077]
98,69,[0.7899891955527619]


In [103]:
def validacion_simple_Parallel_Numpy(Data_Test, X_train, Y_train):

    ##########################

    from joblib import Parallel, delayed
    import multiprocessing

    n_jobs  = multiprocessing.cpu_count()

    ##########################

    def prediction(i, Data_Test, X_train, Y_train ):

     x_new = Data_Test.iloc[ i , range(1 , Data_Test.shape[1])]

     # Usamos KNN_regression_Parallel en vez de KNN_regression
 
     y_new_predict , distances = KNN_regression_Parallel_Numpy( X_train  , Y_train , x_new, k=10, distance = "Gower" , p1=5, p2=2, p3=1  )

     return(y_new_predict)

    ##########################

    y_predictions_vector = []

    # Paralelizamos el siguiente bucle for :

    # for i in  range(0, len(Data_Test)):

        # y_new_predict = prediction(i, Data_Test, X_train, Y_train )

        # y_predictions_vector.append( y_new_predict )

    
    y_predictions_vector = Parallel(n_jobs=n_jobs)( delayed(prediction)( i, Data_Test, X_train, Y_train) for i in range(0, len(Data_Test)) )

    #########################

    ECM = ( (y_predictions_vector - Data_Test.loc[: , 'Y'])**2 ).sum() 


    return(y_predictions_vector, ECM)

In [104]:
y_predictions_vector, ECM = validacion_simple_Parallel_Numpy(Data_Test=Data_Test, X_train=X_train_small , Y_train=Y_train_small)

In [88]:
ECM

3054953133867935.5

La opcion paralelizada y con Numpy solo **26 segundos** , mientras que la parelizada pero con Pandas **1.35 min**, y la no paralelizada y con pandas **5.30 min**

In [105]:
y_predictions_vector, ECM = validacion_simple_Parallel_Numpy(Data_Test=Data_Test, X_train=X_train_medium , Y_train=Y_train_medium)

In [106]:
ECM

1312629089068167.2

En este caso, usando la opcion paralelizada y con Numpy, con las primeras 500 filas de X_train e Y_train , el proceso tarda en ejecutarse **40 minutos**. Recordemos que cuando se hizo lo mismo usando el codigo sin optimizar el proceso fue interrumpido cuando llevaba **mas de 70 minutos** corriendose.

---

## Alternativas a los bucles for <a class="anchor" id="4"></a>

### Método `lambda`

In [89]:
funcion = lambda x, y: x*y

In [90]:
funcion(3, 2)

6

In [91]:
ifelse = lambda x: True  if x >= 10   else False

In [92]:
ifelse(5)

False

In [93]:
ifelse(40)

True

### Función `Map`

Con un argumento:

In [94]:
list_values = [2,4,8,20]

result = map(lambda x: x**2 , list_values)

In [95]:
list(result)

[4, 16, 64, 400]

Con mas de un argumento:

In [96]:
list_values_1 = [2,4,8,20]
list_values_2 = [0,10,2,1]

result = map(lambda x,y: x*y , list_values_1, list_values_2)

In [97]:
list(result)

[0, 40, 16, 20]

### Función `Apply`

In [98]:
X_train.apply(np.mean, axis=0)

size_in_m_2              133.079524
no_of_bedrooms             1.805118
no_of_bathrooms            2.523622
latitude                  25.116866
longitude                 55.210818
private_garden_recode      0.017717
private_pool_recode        0.042651
quality_recode             1.608268
dtype: float64

In [99]:
X_train.apply(np.std, axis=0)

size_in_m_2              84.602381
no_of_bedrooms            0.947825
no_of_bathrooms           1.059314
latitude                  0.062086
longitude                 0.068892
private_garden_recode     0.131919
private_pool_recode       0.202069
quality_recode            0.679246
dtype: float64

---

## Bibliografia

Tomar, A. (29 June 2022). Don’t Run Loops in Python, Instead, Use These!. *medium*.  https://medium.com/codex/3-most-effective-yet-underutilized-functions-in-python-d865ffaca0bb

Amat Rodrigo, J. (November 2020). Paralelizar bucle for con Python. *Cienciadedatos*.  https://www.cienciadedatos.net/documentos/py12-paralelizar-con-python.html